<a href="https://colab.research.google.com/github/aaghash17/price_action_test/blob/main/Price%20Action%20Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
#Add all the imports here
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from itertools import combinations
import itertools


In [56]:
#To get daywise finanace
def get_day_data(symbol, date):
  ticker = yf.Ticker(symbol)
  start_date = datetime.strptime(date, "%Y-%m-%d")
  end_date = start_date + timedelta(days=1)
  try:
    data = ticker.history(start=start_date, end=end_date, interval="1m");
  except Exception as e:
    return None
  return data

In [57]:
#To get range data
def get_range_data(ticker,start_date, end_date):
  start_date = datetime.strptime(start_date, "%Y-%m-%d")
  end_date = datetime.strptime(end_date, "%Y-%m-%d")
  data_list = []
  current_date = start_date
  while current_date <= end_date:
    try:
      data = get_day_data(ticker, current_date.strftime("%Y-%m-%d"))
    except Exception as e:
      return pd.DataFrame()
    if data is not None and not data.empty:
      data_list.append(data)
    current_date += timedelta(days=1)
  if data_list:
    return pd.concat(data_list)
  return pd.DataFrame()

In [58]:
#Main entry
ticker_symbol = "TATAMOTORS.NS"

start_date = "2025-08-26"
end_date = "2025-08-26"

historical_data = get_range_data(ticker_symbol, start_date, end_date)

In [59]:
import plotly.graph_objects as go

# ✅ Only select candles that fall on the hour
hourly_ticks = historical_data.index[historical_data.index.minute == 0]

fig_candle = go.Figure(
    data=[go.Candlestick(
        x=historical_data.index,
        open=historical_data['Open'],
        high=historical_data['High'],
        low=historical_data['Low'],
        close=historical_data['Close'],
        increasing_line_color='green',
        decreasing_line_color='red'
    )]
)

fig_candle.update_layout(
    title=f'{ticker_symbol} Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    dragmode="pan",                # ✅ allow panning
    xaxis=dict(
        type="category",
        rangeslider=dict(visible=False),
        tickmode="array",
        tickvals=hourly_ticks,                      # ✅ positions for ticks
        ticktext=hourly_ticks.strftime("%b %d, %H:00") # ✅ label style: "Aug 10, 11h"
    )
)

# ✅ enable zoom in both directions
fig_candle.update_xaxes(fixedrange=False)
fig_candle.update_yaxes(fixedrange=False)

fig_candle.show(config={"scrollZoom": True})


In [60]:
fig_scatter = go.Figure()

fig_scatter.add_trace(go.Scatter(
    x=historical_data.index,
    y=historical_data['High'],
    mode='markers',
    name='High Prices',
    marker=dict(color='green', size=4)
))

# Add a scatter plot for the Low values using markers
#fig_scatter.add_trace(go.Scatter(
#    x=historical_data.index,
#    y=historical_data['Low'],
#    mode='markers',
#    name='Low Prices',
#    marker=dict(color='red', size=4)
#))

# Customize the chart layout
fig_scatter.update_layout(
    title='High and Low Prices (Markers Only)',
    xaxis_title='Date',
    yaxis_title='Price',
    dragmode="pan",                # ✅ allow panning
    xaxis=dict(
        type="category",
        rangeslider=dict(visible=False),
        tickmode="array",
        tickvals=hourly_ticks,                      # ✅ positions for ticks
        ticktext=hourly_ticks.strftime("%b %d, %H:00") # ✅ label style: "Aug 10, 11h"
    )
)

fig_scatter.show()

In [61]:
from collections import defaultdict
from fractions import Fraction

points = list(enumerate(historical_data["High"].values))

n = len(points)
lines = defaultdict(set)

for i in range(n):
  x1, y1 = points[i]
  for j in range(i+1, n):
    x2, y2 = points[j]
    dx, dy = x2 - x1, y2 - y1

    if dx == 0:
      slope = 'inf'
      intercept = round(x1, 2)  # keep precision limited
    else:
      slope = round(dy/dx, 2)
      intercept = round(y1 - slope*x1, 2)

      key = (slope, intercept)
      lines[key].add((x1, y1))
      lines[key].add((x2, y2))

colinear_groups = []
for pts in lines.values():
  if len(pts) >= 3:
    colinear_groups.append(tuple(sorted(pts)))

# deduplicate
colinear_groups = list(set(colinear_groups))

for g in colinear_groups:
    print(g)

((279, np.float64(683.0999755859375)), (282, np.float64(683.2000122070312)), (284, np.float64(683.25)), (289, np.float64(683.4000244140625)))
((256, np.float64(682.5999755859375)), (258, np.float64(682.5)), (266, np.float64(682.0999755859375)), (267, np.float64(682.0)))
((250, np.float64(681.9000244140625)), (255, np.float64(682.0499877929688)), (265, np.float64(682.3499755859375)), (292, np.float64(683.25)), (293, np.float64(683.0999755859375)))
((302, np.float64(682.3499755859375)), (313, np.float64(682.7000122070312)), (315, np.float64(682.7999877929688)), (330, np.float64(683.1500244140625)), (331, np.float64(683.25)))
((61, np.float64(681.7999877929688)), (64, np.float64(681.9000244140625)), (80, np.float64(682.4000244140625)))
((311, np.float64(681.9500122070312)), (319, np.float64(682.5)), (321, np.float64(682.6500244140625)))
((5, np.float64(681.5)), (6, np.float64(681.3499755859375)), (9, np.float64(680.9000244140625)))
((14, np.float64(680.9500122070312)), (34, np.float64(682